WIP

In [1]:
import torch

class GroupedQueryAttention(torch.nn.Module):
	def __init__(self, heads=8, hidden_size=512):
		super().__init__()
		self.heads = heads
		self.W_k = torch.nn.Linear(hidden_size, hidden_size // heads, bias=False)
		self.W_q = torch.nn.Linear(hidden_size, hidden_size, bias=False)
		self.W_v = torch.nn.Linear(hidden_size, hidden_size // heads, bias=False)
	
	def forward(self, x):
		B, C, H = x.shape
		d = H // self.heads
		k = self.W_k(x).unsqueeze(-3)  # add heads dimension
		q = self.W_q(x).view(B, C, self.heads, d).transpose(-2, -3)
		att = q @ k.transpose(-1, -2)
		att = (att / d**5).softmax(dim=-1)
		v = self.W_v(x).unsqueeze(-3)
		x = (att @ v).transpose(-3, -2).reshape(B, C, H)
		return x

gqa = GroupedQueryAttention()
x = torch.randn(10, 128, 512)
gqa(x).shape

torch.Size([10, 128, 512])